In [1]:
#%pip install ipywidgets

In [25]:
import ipywidgets as widgets
#import a fucntion that will actually display this widget
from IPython.display import display
from threading import Thread 
from queue import Queue #will let us pass messages between threads 

messages = Queue() #this will tell the thread when to stop and recording and when to stop transcribing 
recordings = Queue() #this will actually store the audio data and pass it to the transcription

#we will create our first button 
record_button = widgets.Button(
    description='Record', #text that appears on the button 
    disabled=False, #since we want the button to be enabled
    button_style='success', #this determines the color of the button
    tooltip='Record',
    icon='microphone' #add a little icon to the button that looks like a microphone
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

#we also need an output widget, thats actually gonna show the transcript as it is generated
output = widgets.Output()

def start_recording(data):
    messages.put(True) #tell the thread to keep running and keep recording the microphone 
    
    with output: #context manager
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start() #will start the thread and have it run in the background
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get() #takes the message off the queue
        display("Stopped.")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output) #call the function and this will actually render the widget/button

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

Exception in thread Thread-11 (record_microphone):
Traceback (most recent call last):
  File "C:\Users\asus\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner


In [3]:
#we need to create two functions, one that will run when we click record and other that will run when we click stop button

#RECORDING FROM YOUR MICROPHONE FROM PYAUDIO
#!pip install pyaudio

In [4]:
import pyaudio
p = pyaudio.PyAudio() #initialize pyaudio interface 
#getting info on audio devices on our device 
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (IntelÂ® Smart ', 'hostApi': 0, 'maxInputChannels': 4, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speakers (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels

In [21]:
#index 12, 13

#fucntion that will record our microphone

#constant that we need to define, these define how our audio is going to be recorded, all values are optimal for speech recognition 
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024): #chunk how often we are gonna read from our microphone 
    p = pyaudio.PyAudio()
    #stream this is gonna connect to our microphone and record 
    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=18,
                    frames_per_buffer=chunk) #how often we are read data from our microphone 

    frames = [] #store all our data audio that's recorded from our microphone 

    while not messages.empty():
        data = stream.read(chunk) #we are going to read 1024 audio frames at a time 
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy()) #add our audio data to our recordings queue
            frames = []

    stream.stop_stream() #close the connection 
    stream.close()
    p.terminate()

In [22]:
#pip install vosk

In [23]:
#pip install torch

In [24]:
#RECOGNIZING LIVE SPEECH WITH VOSK--> in order to actually do our speech recognition  
#vosk actually returns its result in json format 
import subprocess #use this to call our punctuation model
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE) #create a recognizer, this will use the model to actually do the speech recognition
rec.SetWords(True) #give us confidence level for each individual words 
    
def speech_recognition(output):
    
    while not messages.empty():#making sure that we have not clicked stop button
        frames = recordings.get() #pulling the data out of the queue 
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"] #to laod the results 
        
        cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(cased) #adding our transcript into our output widget 
        time.sleep(1)

In [9]:
#PROJECT OVERVIEW AND USE CASES

